In [2]:
import random
import numpy as np
import pandas as pd
from citipy import citipy
import matplotlib.pyplot as plt
import requests
from datetime import datetime

# Import API key
from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [4]:
# Add lat long to a list
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
lat_lngs = []
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

773

In [8]:
# list for cities
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
                          

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | atuona
Processing Record 4 of Set 1 | nanortalik
Processing Record 5 of Set 1 | aksum
Processing Record 6 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 7 of Set 1 | coquimbo
Processing Record 8 of Set 1 | tezu
Processing Record 9 of Set 1 | young
Processing Record 10 of Set 1 | puerto ayora
Processing Record 11 of Set 1 | butaritari
Processing Record 12 of Set 1 | sao filipe
Processing Record 13 of Set 1 | qaanaaq
Processing Record 14 of Set 1 | yellowknife
Processing Record 15 of Set 1 | saint george
Processing Record 16 of Set 1 | souillac
Processing Record 17 of Set 1 | hithadhoo
Processing Record 18 of Set 1 | hobart
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | olafsvik
Processing Record 21 of Set 1 | hasaki
Processing Record

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,50.11,66,0,23.02,clear sky
1,Atuona,PF,-9.8000,-139.0333,78.06,77,81,18.63,light rain
2,Nanortalik,GL,60.1432,-45.2371,36.05,86,97,5.88,overcast clouds
3,Aksum,ET,14.1245,38.7244,58.17,36,24,4.43,few clouds
4,Coquimbo,CL,-29.9533,-71.3436,64.13,63,0,14.97,clear sky
5,Tezu,IN,27.9167,96.1667,73.65,81,51,4.29,broken clouds
6,Young,UY,-32.6833,-57.6333,72.23,47,1,7.40,clear sky
7,Puerto Ayora,EC,-0.7393,-90.3518,78.76,84,96,7.00,light rain
8,Butaritari,KI,3.0707,172.7902,81.41,73,12,12.12,few clouds
9,Sao Filipe,CV,14.8961,-24.4956,78.73,78,0,10.40,clear sky


In [10]:
# Create the output file CSV
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export DF
city_data_df.to_csv(output_data_file, index_label="City_ID")